In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/lab3.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
from sklearn.model_selection import train_test_split

# Creating similar size samples of ham and spam
ham = df[df['Category'] == 'ham']
spam = df[df['Category'] == 'spam']
ham = ham.sample(spam.shape[0])

# Now combine these to form a dataset
dataset = pd.concat([ham, spam], ignore_index=True)

# Split for training and testing 
X_train, X_test, y_train, y_test = train_test_split(dataset['Message'], dataset['Category'], test_size=0.3)

# Convert to numeric vector format for model
# Integer feature counts work better than fractional counts like tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
ctvector = TfidfVectorizer()
X_train = ctvector.fit_transform(X_train).toarray()
X_test = ctvector.transform(X_test).toarray()

In [21]:
# A1 
# K-means clustering with k = 3 and k = 5
# Silhoette score is a metric used to evaluate how good clustering results are  
# in data clustering 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def kmeans(n, flag = False):
    # Training model and fitting training data
    model = KMeans(n_clusters=n, random_state=0, n_init="auto")
    model.fit(X_train)

    # Predicting labels for test data
    labels = model.predict(X_test)

    if flag == True:
        # Testing accuracy of clustering model
        silhouette_avg = silhouette_score(X_test, labels)
    return {"cluster":n, "accuracy_score":silhouette_avg} if flag == True else None


# Number of clusters
num_of_clusters = 3
print(kmeans(3, True))
print(kmeans(5, True))

{'cluster': 3, 'accuracy_score': 0.007776861115919134}
{'cluster': 5, 'accuracy_score': 0.00959434605733763}


In [ ]:
# A2 
# Determine the k-value for your dataset; k = {1, 31}
# Use elbow method to determine the ideal value of k based on average Euclidean
# distance from cluster center